In [99]:
!pip install beautifulsoup4

In [ ]:
import csv
import pandas as pd

In [ ]:
urls = []
with open('newsUrls1.csv', 'r') as file:
  reader = csv.reader(file)
  for row in reader:
    urls.extend(row)

with open('newsUrls.csv', 'r') as file:
  reader = csv.reader(file)
  for row in reader:
    urls.extend(row)


In [ ]:
newsSources = {
    "center":[
        "site:https://politicalwire.com/",
        "site:https://thehill.com/ -inurl:/social-tags AND -inurl:/tag",
        "site:https://www.livemint.com/ -inurl:/topic",
        "site:https://www.ft.com/content",
        "site:https://www.japantimes.co.jp/",
        "site:https://wtvbam.com/",
        "site:https://san.com/ -inurl:/media-miss",
        "site:https://bnonews.com/",
        "site:https://ny1.com/"],
    "left":[
        "site:https://www.independent.co.uk/",
        "site:https://asiatimes.com/",
        "site:https://www.gzeromedia.com/",
        "site:https://www.cnn.com/",
        "site:https://japantoday.com/",
        "site:https://www.cbc.ca/ -inurl:/player",
        "site:https://www.nbcnews.com/",
        "site:https://apnews.com/ -inurl:/video",
        "site:https://deadline.com/",
        "site:https://www.npr.org/",
        "site:https://www.usatoday.com/",
        "site:https://www.rawstory.com/"
    ],
    "right":[
        "site:https://www.republicworld.com/",
        "site:https://newsaddicts.com/",
        "site:https://www.theepochtimes.com/",
        "site:https://americafirstreport.com/",
        "site:https://trendingpoliticsnews.com/",
        "site:https://www.wnd.com/",
        "site:https://www.washingtontimes.com/",
        "site:https://www.foxnews.com/",
        "site:https://headlines.americanlookout.com/",
        "site:https://www.zerohedge.com/"
    ]
}

In [117]:
# prompt: Write code to read content from urls and store it in a dictionary

import csv
import requests
import uuid

from bs4 import BeautifulSoup
import re

def getView(url):
  center = ['politicalwrite','thehill','livemint','ft.com','japantimes','wtvbam','san','bnonews','ny1']
  left = ['independent','asiatimes','gzeromedia','cnn','japantoday','cbc','nbcnews','apnews','deadline','npr','usatoday','rawstory']
  right = ['republicworld','newsaddicts','theepochtimes','americafirstreport','trendingpoliticsnews','wnd','washingtontimes','foxnews','americanlookout','zerohedge']
  if any(site for site in center if site in url):
    return "center"
  elif any(site  for site in left if site in url):
    return "left"
  elif any(site  for site in right if site in url):
    return "right"
  else:
    return "other"


def fetch_content_from_urls(urls):
    url_content_list = []
    i=0
    for url in urls:
      i+=1
      try:
          if ".pdf" in url or "cbc.ca" in url:
            print(f"{i} - Skipping PDF URL: {url}")
            continue
          print(f"{i} - Fetching URL: {url}")
          response = requests.get(url,timeout=30)
          response.raise_for_status()  # Raise HTTPError for bad responses (4xx or 5xx)
          soup = BeautifulSoup(response.content, "html.parser")
          text = soup.get_text(separator=' ', strip=True)
          text = re.sub(r'[\n\t]+', ' ', text)
          text = re.sub(r'\s+', ' ', text).strip()
          url_content_list.append({"content":text,"view":getView(url),"url":url})
      except requests.exceptions.RequestException as e:
          print(f"Error fetching URL {url}: {e}")
          #url_content_dict[url] = "Error fetching content"  # Store error message

    return url_content_list




In [118]:
url_content_list = fetch_content_from_urls(urls)

1 - Fetching URL: https://www.cnn.com/2022/04/07/politics/lgbtq-rights-movement-race-deconstructed-newsletter/index.html
2 - Fetching URL: https://www.independent.co.uk/news/world/americas/us-politics/jen-psaki-abortion-transgender-arizona-oklahoma-b2048557.html
Error fetching URL https://www.independent.co.uk/news/world/americas/us-politics/jen-psaki-abortion-transgender-arizona-oklahoma-b2048557.html: 405 Client Error: Not allowed. for url: https://www.independent.co.uk/news/world/americas/us-politics/jen-psaki-abortion-transgender-arizona-oklahoma-b2048557.html
3 - Fetching URL: https://ny1.com/nyc/queens/politics/2024/10/22/south-korea-warns-it-can-send-arms-to-ukraine-after-reports-of-north-s-troops-in-russia
4 - Fetching URL: https://www.independent.co.uk/news/world/americas/us-politics/national-dont-say-gay-mike-johnson-b2206438.html
Error fetching URL https://www.independent.co.uk/news/world/americas/us-politics/national-dont-say-gay-mike-johnson-b2206438.html: 405 Client Error

In [119]:
df = pd.DataFrame(url_content_list)
df

,content,view,url
0,Historic Supreme Court confirmation comes at a...,left,https://www.cnn.com/2022/04/07/politics/lgbtq-...
1,South Korea warns it can send arms to Ukraine ...,center,https://ny1.com/nyc/queens/politics/2024/10/22...
2,CNN Site Map (Articles) for March - 2023 CNN S...,left,https://www.cnn.com/intl_planning/article/site...
3,More than 200 medical professionals from nearl...,left,https://www.cnn.com/2020/02/07/health/doctors-...
4,"Bud Light wanted to market to all. Now, it’s a...",left,https://www.cnn.com/2023/05/01/business/bud-li...
...,...,...,...
383,Missouri voters approve abortion rights in sta...,left,https://apnews.com/article/abortion-missouri-a...
384,Abortion-rights ballot measures pass in 7 stat...,left,https://www.nbcnews.com/politics/2024-election...
385,Abortion rights advocates say they need more m...,left,https://japantoday.com/category/features/opini...
386,Associated Press Sues Trump Administration Ove...,left,https://deadline.com/2025/02/trump-associated-...


In [120]:
df.to_csv('extractedNews.csv', index=False) # Save as CSV without the index